<a href="https://colab.research.google.com/github/AntunesLab/crossdome/blob/cd_on_colab/CrossDome_Multiple_entry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dinlerantunes.com/assets/images/lab-logo.png" height="200">


# Welcome to CrossDome on Google Colab!

---

The full CrossDome source and documentation are available at https://github.com/AntunesLab/crossdome

---

***If you use CrossDome in published research, please cite:***
Fonseca AF, Antunes DA. CrossDome: an interactive R package to predict cross-reactivity risk using immunopeptidomics databases. Front Immunol. 2023;14:1142573. Published 2023 Jun 12. doi:10.3389/fimmu.2023.1142573

---

**Acknowledgments**
This Google Colab Notebook was created by Antunes Lab: Martiela Freitas and Pamella Borges.

In [2]:
import os

# Check if the file exists before attempting to copy or extract
if not os.path.exists('usr/local/lib/R/site-library/crossdome'):
  os.system(f'wget https://www.dropbox.com/scl/fi/yxghqpynw0bo8gouic694/library_1.zip?rlkey=7mafkhdhkn86kf9lovkt7gi06&st=81kwlz0z&dl=0')
  os.system(f'unzip /content/library_1.zip?rlkey=7mafkhdhkn86kf9lovkt7gi06')
else:
  print("File already exists. Skipping download and extraction.")

File already exists. Skipping download and extraction.


In [3]:
%%capture
# activate R magic
%load_ext rpy2.ipython

In [4]:
%%R

if(file.exists('/content/usr/local/lib/R/site-library/crossdome')){
    #Load library
  .libPaths('usr/local/lib/R/site-library/')
  library(crossdome)
}
else{
  print("CrossDome library not found yet. Please, wait a few minuts to finish the library extraction and run again")
}


If you use crossdome in published research, please cite: <coming soon>




Beware that CrossDome only works for 9mer peptides.

You chose to run CrossDome using a table as input, please use the following format:

|    Allele    |    Peptide   |
| ------------ | ------------ |
| HLA-A*02:01  | RPILTISTL    |
| HLA-B*07:02  | RPILTISTL    |

In [20]:
# upload your table to /content/

from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [13]:
%%R

# Please change the name of your table here.
df <- read.csv("/content/test.csv")

# Using data from CSV file to set variables
for (i in 1:nrow(df)) {
  allele_value <- df[i, 1]
  query_value <- df[i, 2]

  # Create a unique identifier for files
  file_identifier <- paste(allele_value, query_value, sep = "_")
  file_identifier <- gsub("[:*]", "", file_identifier) # Remove special characters that may not be valid in file names

  # Set up the database with specific allele, withOUT off_target
  tryCatch({
    database <- cross_background(allele = allele_value)

    # Compose result using the query value
    result <- cross_compose(query = query_value, background = database)

    # Save each result to a separate CSV file
    result_file_name <- paste0(file_identifier, ".csv")
    write.csv(result@result, file = result_file_name, row.names = FALSE)
    print(paste0("Your results were saved as: ", result_file_name))

    # Extracting the gene donor mRNA expression based on CR candidates
    expression_matrix <- cross_expression_matrix(result, pvalue_threshold = 0.005)

    # The heatmap presenting the gene donor expression profile
    pdf(paste0(file_identifier, "_expression_heatmap.pdf"), width = 10, height = 8)
    cross_expression_plot(object = expression_matrix)
    on.exit(dev.off())
    print(paste0("Your heatmap presenting the gene donor expression profile was saved as: ", file_identifier, "_expression_heatmap.pdf"))

    # Bar plot summarizing the tissue-specificity groups
    pdf(paste0(file_identifier, "_tissue_specificity.pdf"), width = 10, height = 8)
    cross_tissues_plot(object = expression_matrix)
    on.exit(dev.off())
    print(paste0("Your Bar plot summarizing the tissue-specificity groups was saved as: ", file_identifier, "_tissue_specificity.pdf"))

    # Calculate position-specific substitution across cross-reactive candidates
    cross_result <- cross_substitution_matrix(result)

    # Heatmap combined with seqlogo displaying amino acid substitutions
    pdf(paste0(file_identifier, "_cross_substitution.pdf"), width = 10, height = 8)
    cross_substitution_plot(object = cross_result)
    on.exit(dev.off())
    print(paste0("Your Heatmap combined with seqlogo displaying amino acid substitutions was saved as: ", file_identifier, "_cross_substitution.pdf"))

  }, error = function(e) {
    # Handle error by printing a message
    print(paste0("Error processing allele '", allele_value, "': ", e$message))
  })
}

##------ Tue Aug 27 15:27:40 2024 ------##
[1] "Your results were saved as: HLA-A0201_RPILTISTL.csv"
[1] "Your heatmap presenting the gene donor expression profile was saved as: HLA-A0201_RPILTISTL_expression_heatmap.pdf"
[1] "Your Bar plot summarizing the tissue-specificity groups was saved as: HLA-A0201_RPILTISTL_tissue_specificity.pdf"
[1] "Your Heatmap combined with seqlogo displaying amino acid substitutions was saved as: HLA-A0201_RPILTISTL_cross_substitution.pdf"
##------ Tue Aug 27 15:27:51 2024 ------##
[1] "Your results were saved as: HLA-B0702_RPILTISTL.csv"
[1] "Your heatmap presenting the gene donor expression profile was saved as: HLA-B0702_RPILTISTL_expression_heatmap.pdf"
[1] "Your Bar plot summarizing the tissue-specificity groups was saved as: HLA-B0702_RPILTISTL_tissue_specificity.pdf"
[1] "Your Heatmap combined with seqlogo displaying amino acid substitutions was saved as: HLA-B0702_RPILTISTL_cross_substitution.pdf"


In [ ]:
#@title ### Import Google Drive
#@markdown Click on the "Run" button if you would like to connect your analysis to your Google Drive.
%%capture


In [ ]:
#@title ### **Save results on Drive**

#@markdown Run this cell to save your results on your Google Drive
#@markdown ---
#@markdown Please, complete the name of the folder you would like to create with your results in the folder_name area

from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

folder_name = "CrossDome_Results" #@param {type:"string"}

#@markdown Please, complete the path of the folder you would like to create with your results in the path_results area
path_results = "/content/drive/MyDrive/1.PhD/CrossDome" #@param {type:"string"}
# Import necessary modules
import os

# Construct the full path to the results
full_results_path = os.path.join(path_results, folder_name)


# Create a folder in the root directory
if os.path.exists(path_results):
    # Create the folder to save the results
    os.system(f'mkdir -p {full_results_path}')

    #directory_files = os.listdir("/content/")
    os.system(f'cp /content/*.pdf {full_results_path}')
    os.system(f'cp /content/*.csv {full_results_path}')
    print("Results saved successfully.")
else:
    print("Folder not found. Please check the library path and name.")



Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Results saved successfully.


**If any of your heatmaps seems o tight, please run Colabe_Single_Entry for the respective pair of Allele and Peptide.**